# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 28 2023 8:30AM,258296,10,0086452054,ISDIN Corporation,Released
1,Feb 28 2023 8:30AM,258296,10,0086452055,ISDIN Corporation,Released
2,Feb 28 2023 8:30AM,258296,10,0086452056,ISDIN Corporation,Released
3,Feb 28 2023 8:30AM,258296,10,0086452053,ISDIN Corporation,Released
4,Feb 28 2023 8:30AM,258296,10,0086452057,ISDIN Corporation,Released
5,Feb 28 2023 8:30AM,258296,10,0086451155,ISDIN Corporation,Released
6,Feb 28 2023 8:30AM,258296,10,0086451151,ISDIN Corporation,Released
7,Feb 28 2023 8:30AM,258296,10,0086451150,ISDIN Corporation,Released
8,Feb 28 2023 8:30AM,258296,10,0086451619,ISDIN Corporation,Released
9,Feb 28 2023 8:30AM,258296,10,0086451623,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
49,258294,Released,1
50,258295,Released,1
51,258296,Released,57
52,258297,Released,4
53,258298,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
258294,NaN,NaN,1.0
258295,NaN,NaN,1.0
258296,NaN,NaN,57.0
258297,NaN,NaN,4.0
258298,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258153,0.0,31.0,0.0
258162,0.0,0.0,58.0
258166,13.0,4.0,0.0
258188,0.0,24.0,0.0
258201,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258153,0,31,0
258162,0,0,58
258166,13,4,0
258188,0,24,0
258201,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258153,0,31,0
1,258162,0,0,58
2,258166,13,4,0
3,258188,0,24,0
4,258201,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258153,,31,
1,258162,,,58
2,258166,13,4,
3,258188,,24,
4,258201,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 28 2023 8:30AM,258296,10,ISDIN Corporation
53,Feb 28 2023 8:30AM,258291,10,ISDIN Corporation
104,Feb 28 2023 8:30AM,258290,10,"Snap Medical Industries, LLC"
114,Feb 28 2023 8:30AM,258293,10,"Snap Medical Industries, LLC"
115,Feb 28 2023 8:30AM,258294,10,"Senseonics, Incorporated"
116,Feb 27 2023 3:26PM,258298,22,"NBTY Global, Inc."
117,Feb 27 2023 3:25PM,258297,19,"GCH Granules USA, Inc."
121,Feb 27 2023 3:22PM,258295,19,"Snap Medical Industries, LLC"
122,Feb 27 2023 3:11PM,258292,16,"SHL Pharma, LLC"
123,Feb 27 2023 3:00PM,258287,12,"SD Head USA, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 28 2023 8:30AM,258296,10,ISDIN Corporation,,,57
1,Feb 28 2023 8:30AM,258291,10,ISDIN Corporation,,,47
2,Feb 28 2023 8:30AM,258290,10,"Snap Medical Industries, LLC",,,10
3,Feb 28 2023 8:30AM,258293,10,"Snap Medical Industries, LLC",,,1
4,Feb 28 2023 8:30AM,258294,10,"Senseonics, Incorporated",,,1
5,Feb 27 2023 3:26PM,258298,22,"NBTY Global, Inc.",,,1
6,Feb 27 2023 3:25PM,258297,19,"GCH Granules USA, Inc.",,,4
7,Feb 27 2023 3:22PM,258295,19,"Snap Medical Industries, LLC",,,1
8,Feb 27 2023 3:11PM,258292,16,"SHL Pharma, LLC",,,1
9,Feb 27 2023 3:00PM,258287,12,"SD Head USA, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 28 2023 8:30AM,258296,10,ISDIN Corporation,57,,
1,Feb 28 2023 8:30AM,258291,10,ISDIN Corporation,47,,
2,Feb 28 2023 8:30AM,258290,10,"Snap Medical Industries, LLC",10,,
3,Feb 28 2023 8:30AM,258293,10,"Snap Medical Industries, LLC",1,,
4,Feb 28 2023 8:30AM,258294,10,"Senseonics, Incorporated",1,,
5,Feb 27 2023 3:26PM,258298,22,"NBTY Global, Inc.",1,,
6,Feb 27 2023 3:25PM,258297,19,"GCH Granules USA, Inc.",4,,
7,Feb 27 2023 3:22PM,258295,19,"Snap Medical Industries, LLC",1,,
8,Feb 27 2023 3:11PM,258292,16,"SHL Pharma, LLC",1,,
9,Feb 27 2023 3:00PM,258287,12,"SD Head USA, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 28 2023 8:30AM,258296,10,ISDIN Corporation,57,,
1,Feb 28 2023 8:30AM,258291,10,ISDIN Corporation,47,,
2,Feb 28 2023 8:30AM,258290,10,"Snap Medical Industries, LLC",10,,
3,Feb 28 2023 8:30AM,258293,10,"Snap Medical Industries, LLC",1,,
4,Feb 28 2023 8:30AM,258294,10,"Senseonics, Incorporated",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 28 2023 8:30AM,258296,10,ISDIN Corporation,57.0,NaN,NaN
1,Feb 28 2023 8:30AM,258291,10,ISDIN Corporation,47.0,NaN,NaN
2,Feb 28 2023 8:30AM,258290,10,"Snap Medical Industries, LLC",10.0,NaN,NaN
3,Feb 28 2023 8:30AM,258293,10,"Snap Medical Industries, LLC",1.0,NaN,NaN
4,Feb 28 2023 8:30AM,258294,10,"Senseonics, Incorporated",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 28 2023 8:30AM,258296,10,ISDIN Corporation,57.0,0.0,0.0
1,Feb 28 2023 8:30AM,258291,10,ISDIN Corporation,47.0,0.0,0.0
2,Feb 28 2023 8:30AM,258290,10,"Snap Medical Industries, LLC",10.0,0.0,0.0
3,Feb 28 2023 8:30AM,258293,10,"Snap Medical Industries, LLC",1.0,0.0,0.0
4,Feb 28 2023 8:30AM,258294,10,"Senseonics, Incorporated",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5939667,367.0,80.0,0.0
12,516532,1.0,1.0,0.0
15,774611,32.0,29.0,83.0
16,774765,2.0,1.0,0.0
18,258279,1.0,0.0,0.0
19,2582434,32.0,20.0,13.0
20,258284,23.0,0.0,0.0
21,258234,1.0,0.0,0.0
22,774808,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5939667,367.0,80.0,0.0
1,12,516532,1.0,1.0,0.0
2,15,774611,32.0,29.0,83.0
3,16,774765,2.0,1.0,0.0
4,18,258279,1.0,0.0,0.0
5,19,2582434,32.0,20.0,13.0
6,20,258284,23.0,0.0,0.0
7,21,258234,1.0,0.0,0.0
8,22,774808,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,367.0,80.0,0.0
1,12,1.0,1.0,0.0
2,15,32.0,29.0,83.0
3,16,2.0,1.0,0.0
4,18,1.0,0.0,0.0
5,19,32.0,20.0,13.0
6,20,23.0,0.0,0.0
7,21,1.0,0.0,0.0
8,22,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,367.0
1,12,Released,1.0
2,15,Released,32.0
3,16,Released,2.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,21,22
Status,,,,,,,,,
Completed,0.0,0.0,83.0,0.0,0.0,13.0,0.0,0.0,0.0
Executing,80.0,1.0,29.0,1.0,0.0,20.0,0.0,0.0,0.0
Released,367.0,1.0,32.0,2.0,1.0,32.0,23.0,1.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,21,22
0,Completed,0.0,0.0,83.0,0.0,0.0,13.0,0.0,0.0,0.0
1,Executing,80.0,1.0,29.0,1.0,0.0,20.0,0.0,0.0,0.0
2,Released,367.0,1.0,32.0,2.0,1.0,32.0,23.0,1.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,21,22
0,Completed,0.0,0.0,83.0,0.0,0.0,13.0,0.0,0.0,0.0
1,Executing,80.0,1.0,29.0,1.0,0.0,20.0,0.0,0.0,0.0
2,Released,367.0,1.0,32.0,2.0,1.0,32.0,23.0,1.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()